In [1]:
import networkx as nx;
#import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;

In [2]:
def solver_optimistic (network, budget, rate, T_Limit, summaryName):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';
    
    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                       cost=data.iat[i,3], special=data.iat[i,4], trafficker=data.iat[i,5], 
                       bottom=data.iat[i,6], victim=data.iat[i,7]);
    
    for i,j in G.edges:           # Applying rate to trafficker's capacity (HT Networks)
        if i == s:
            G.edges[i,j]['capacity'] = math.floor(rate*G.edges[i,j]['capacity']);
            
    A = 0;   # Number of special arcs
    T = 0;   # Number of Traffickers 
    B = 0;   # Number of intermediaries (Bottoms)
    V = 0;   # Number of Victims
    U = 0;   # Maximum capacity
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
        if G.edges[i,j]['trafficker'] == 1:
            T = T + 1;
        elif G.edges[i,j]['bottom'] == 1:
            B = B + 1;
        elif G.edges[i,j]['victim'] == 1:
            V = V + 1;
            U = U + G.edges[i,j]['capacity'];
            
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###

    
    ### Optimization Model ###
    
    instance = 'LogFile_'+network+'_b'+str(budget)+'_r'+str(rate)+'.txt';
    
    model = gp.Model("Optimistic"); 
    
    gamma = model.addVars(G.edges, vtype=GRB.BINARY); 
    z = model.addVars(G.edges, vtype=GRB.BINARY); 
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    theta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    alpha = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    beta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    
    
    model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    model.addConstr(gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges) <= x[t,s]);
    
    model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            model.addConstr(x[i,j] - G.edges[i,j]['capacity']*z[i,j] <= 0);
        
        model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
            
    model.setObjective(gp.quicksum(G.edges[i,j]['special']*z[i,j] for i,j in G.edges), GRB.MINIMIZE);
    
    model.setParam("IntegralityFocus",1);
    #model.setParam("MIPFocus", 3);
    #model.setParam('NodefileStart', 0.5)    # Memore Issues
    #model.setParam("NumericFocus",2);
    model.setParam('TimeLimit', T_Limit); 
    model.update();
    
    model.setParam("LogToConsole", 0)
    model.setParam("OutputFlag", 1);
    model.setParam("LogFile", instance);
    model.optimize();
    
    status = model.status;
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = round(model.objVal);
    LB = model.objBound;
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    ### Print to the screen and to result file #######################
    
    file = open('HT_Optimistic_'+network+'_b'+str(budget)+'_r'+str(rate)+'.txt', "w");
    file.write('Instance: %s, budget %g, rate %g \n' %(network, budget, rate));
    file.write('Instance executed on:\t %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g' % (T+B+V) +'\n');
    file.write('Number of Traffickers: %g' % T +'\n');
    file.write('Number of Bottoms: %g' % B +'\n');
    file.write('Number of victims: %g' % (V) +'\n');
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n')
    print('Instance: %s, budget %g, rate %g \n' %(network, budget, rate));
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g' % status);
        file.write('Status: %g \n' % status);
    
    print('\nMax-Flow: %g' %x[t,s].x);
    file.write('Max-Flow: %g \n' %x[t,s].x);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    print('Intediction:')
    for i,j in G.edges:
        if gamma[i,j].x > 0.0001:
            print("arc (%g,%g), gamma = %g " %(i,j, gamma[i,j].x));
    print('\n');
    
    file.write('\nTrafficker Capacity: \n');
    for i,j in G.edges:
        if i==s:
            file.write('(%g, %g) : %g \n' %(i,j,G.edges[i,j]['capacity']));
    
    int_T = 0;
    int_B = 0;
    int_V = 0;
    file.write('\nInterdiction plan: \n');
    for i, j in G.edges: 
        if gamma[i,j].x > 0.0001: 
            if G.edges[i,j]['trafficker'] == 1:
                file.write('Trafficker: %g, gamma = %g \n' %(j, gamma[i,j].x));
                int_T = int_T + 1;
            elif G.edges[i,j]['bottom'] == 1:
                file.write('Bottom: %g, gamma = %g \n'%(j, gamma[i,j].x));
                int_B = int_B + 1;
            elif G.edges[i,j]['victim'] == 1:
                file.write('Victim: %g, gamma = %g \n' %(i, gamma[i,j].x));
                int_V = int_V + 1;
    
    file.write('\nVictim nodes with flow \n');    
    for i, j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if x[i,j].x > 0.0001:
                file.write('victim: %g, \t Flow: %g \n' %(i, x[i,j].x));
    
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, rate, T+B+V, T, B, V, obj, OptGap, x[t,s].x, int_T, int_B, int_V, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();

In [3]:
n_Networks = 10;
Budget = [0, 4, 8, 12, 16, 20, 24];
Rate = [1, 0.75, 0.5];

T_Limit = 3600;

summaryName = "HT_Optimistic_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance\t, Budget\t, Rate\t, Nodes\t, Traffick\t, Bottoms\t, Victims\t,');
file_summary.write('Obj_value\t, OptGap\t, Flow\t, Int_Traf\t, Int_Bot\t, Int_Vic\t,  Run_Time (sec)\n');
file_summary.close();

for n in range(1, n_Networks+1):
    network = 'Network'+str(n);
    
    for budget in Budget:
        
        for rate in Rate:
            solver_optimistic (network, budget, rate, T_Limit, summaryName);

Academic license - for non-commercial use only - expires 2024-01-14
Using license file /home/daniel/gurobi.lic
Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network1, budget 0, rate 1 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 36

Max-Flow: 195
run time: 0.09 sec 

Intediction:


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network1, budget 0, rate 0.75 

Status: Optimal
Number of special arcs without flow: 16
Number of special arcs with flow: 20

Max-Flow: 144
run time: 0.06 sec 

Intediction:


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min:

Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network1, budget 24, rate 0.5 

Status: Optimal
Number of special arcs without flow: 31
Number of special arcs with flow: 5

Max-Flow: 25
run time: 0.13 sec 

Intediction:
arc (0,1), gamma = 1 
arc (0,11), gamma = 1 
arc (0,20), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network2, budget 0, rate 1 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 27

Max-Flow: 161
run time: 0.06 sec 

Intediction:


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Netw

Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network2, budget 24, rate 0.75 

Status: Optimal
Number of special arcs without flow: 24
Number of special arcs with flow: 3

Max-Flow: 24
run time: 0.17 sec 

Intediction:
arc (0,9), gamma = 1 
arc (0,24), gamma = 1 
arc (2,33), gamma = 1 
arc (3,33), gamma = 1 
arc (22,33), gamma = 1 
arc (23,33), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network2, budget 24, rate 0.5 

Status: Optimal
Number of special arcs without flow: 25
Number of special arcs with flow: 2

Max-Flow: 15
run time: 0.13 sec 

Intediction:
arc (0,4), gamma = 1 
arc (0,9), gamma = 1 
arc (0,24), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev:

   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network3, budget 24, rate 1 

Status: Optimal
Number of special arcs without flow: 19
Number of special arcs with flow: 6

Max-Flow: 41
run time: 0.1 sec 

Intediction:
arc (0,1), gamma = 1 
arc (0,9), gamma = 1 
arc (0,24), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network3, budget 24, rate 0.75 

Status: Optimal
Number of special arcs without flow: 21
Number of special arcs with flow: 4

Max-Flow: 30
run time: 0.04 sec 

Intediction:
arc (0,1), gamma = 1 
arc (0,9), gamma = 1 
arc (0,24), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf

   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network4, budget 20, rate 0.5 

Status: Optimal
Number of special arcs without flow: 27
Number of special arcs with flow: 5

Max-Flow: 41
run time: 0.1 sec 

Intediction:
arc (0,9), gamma = 1 
arc (0,22), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network4, budget 24, rate 1 

Status: Optimal
Number of special arcs without flow: 23
Number of special arcs with flow: 9

Max-Flow: 48
run time: 0.16 sec 

Intediction:
arc (0,1), gamma = 1 
arc (0,9), gamma = 1 
arc (0,22), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network4, 

...

Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network6, budget 20, rate 1 

Status: Optimal
Number of special arcs without flow: 19
Number of special arcs with flow: 13

Max-Flow: 64
run time: 0.15 sec 

Intediction:
arc (0,6), gamma = 1 
arc (0,20), gamma = 1 
arc (29,30), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network6, budget 20, rate 0.75 

Status: Optimal
Number of special arcs without flow: 24
Number of special arcs with flow: 8

Max-Flow: 56
run time: 0.17 sec 

Intediction:
arc (0,6), gamma = 1 
arc (0,20), gamma = 1 
arc (4,38), gamma = 1 
arc (3,38), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance

   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network7, budget 16, rate 0.5 

Status: Optimal
Number of special arcs without flow: 22
Number of special arcs with flow: 6

Max-Flow: 37
run time: 0.09 sec 

Intediction:
arc (0,17), gamma = 1 
arc (0,26), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network7, budget 20, rate 1 

Status: Optimal
Number of special arcs without flow: 18
Number of special arcs with flow: 10

Max-Flow: 62
run time: 0.14 sec 

Intediction:
arc (0,17), gamma = 1 
arc (0,26), gamma = 1 
arc (1,2), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Networ

Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network8, budget 16, rate 0.75 

Status: Optimal
Number of special arcs without flow: 18
Number of special arcs with flow: 7

Max-Flow: 51
run time: 0.09 sec 

Intediction:
arc (0,8), gamma = 1 
arc (0,17), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network8, budget 16, rate 0.5 

Status: Optimal
Number of special arcs without flow: 21
Number of special arcs with flow: 4

Max-Flow: 34
run time: 0.1 sec 

Intediction:
arc (0,8), gamma = 1 
arc (0,17), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Defau

Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network9, budget 16, rate 1 

Status: Optimal
Number of special arcs without flow: 12
Number of special arcs with flow: 9

Max-Flow: 32
run time: 0.19 sec 

Intediction:
arc (0,11), gamma = 1 
arc (0,18), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network9, budget 16, rate 0.75 

Status: Optimal
Number of special arcs without flow: 17
Number of special arcs with flow: 4

Max-Flow: 23
run time: 0.08 sec 

Intediction:
arc (0,11), gamma = 1 
arc (0,18), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Defa

Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network10, budget 12, rate 0.5 

Status: Optimal
Number of special arcs without flow: 20
Number of special arcs with flow: 6

Max-Flow: 45
run time: 0.05 sec 

Intediction:
arc (0,15), gamma = 1 
arc (24,32), gamma = 1 
arc (25,32), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Instance: Network10, budget 16, rate 1 

Status: Optimal
Number of special arcs without flow: 13
Number of special arcs with flow: 13

Max-Flow: 78
run time: 0.17 sec 

Intediction:
arc (0,8), gamma = 1 
arc (0,15), gamma = 1 


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 3600.0
   Prev: inf